# Mr.Chang landing to Venice 

### Coursera Applied Data Science Capstone - Final

In [4]:
import pandas as pd
import numpy as np 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes
import folium 
print('Libraries imported.')

Libraries imported.


In [5]:
df = pd.read_csv("Sestieri.csv", index_col=0)


In [6]:
df

,Latitude,Longitude
Sestiere,,
Cannaregio,45.446765,12.326153
Castello,45.436255,12.343054
Dorsoduro,45.430922,12.324928
Santa Croce,45.441417,12.326927
San Marco,45.434238,12.339009
San Polo,45.437937,12.334652


In [ ]:
map_venice = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, local in zip(df3['Latitude'], df3['Longitude'], df3['Sestieri']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_venice)  
    
map_venice

In [ ]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Sestiere', 
                  'Sestiere Latitude', 
                  'Sestiere Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [ ]:
limit = 500 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius
CLIENT_ID = 'HWJ2TJSH2DDY5LMTBAU5DZMRSLYKWYNLONBTZWEKLNO1J51Y'
CLIENT_SECRET = 'YV2CJBQVCQWKKOSJ0IH0ZYR12CHAW1L0TFEEOYQXRHB4U3GI'
VERSION = '20181020'

In [ ]:
venice_venues_asian = getNearbyVenues(names=df3['Sestieri'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d16c941735')
venice_venues_asian.head()

In [ ]:
venice_venues_asian.shape

In [ ]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Localidad'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [ ]:
map_venice_asian = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(venice_venues_asian, 'red', map_venice_asian)
map_venice_asian

In [ ]:
venice_venues_hotels = getNearbyVenues(names=df3['Sestieri'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d13d941735')
venice_venues_hotels.head()

In [ ]:
venice_venues_hotels.shape

In [ ]:
map_venice_hotels = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(venice_venues_hotels, 'green', map_venice_hotels)
map_venice_hotels

In [ ]:
venice_venues_transport = getNearbyVenues(names=df3['Sestieri'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1ae941735')
venice_venues_transport.head()

In [ ]:
venice_venues_transport.shape

In [ ]:
map_venice_transport = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(venice_venues_transport, 'gold', map_venice_transport)
map_venice_transport

In [ ]:
venice_venues_schools = getNearbyVenues(names=df3['Localidades'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4d4b7105d754a06375d81259')
venice_venues_schools.head()

In [ ]:
venice_venues_schools.shape

In [ ]:
map_venice_schools = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(venice_venues_schools, 'fuchsia', map_venice_schools)
map_venice_schools

In [ ]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Sestiere').count()
    
    for n in startDf['Sestiere']:
        try:
            startDf.loc[startDf['Sestiere'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Sestiere'] == n,columnTitle] = 0

In [ ]:
df_data = df3.copy()
df_data.rename(columns={'Sestieri':'Sestiere'}, inplace=True)
addColumn(df_data, 'Restaurant', venice_venues_asian)
addColumn(df_data, 'Asian Schools', venice_venues_schools)
addColumn(df_data, 'Transport Services', venice_venues_transport)
addColumn(df_data, 'Hotels', venice_venues_hotels)
df_data

In [ ]:
# negative weight, because Jeronimo wants to open a burger joint and thus wants to avoid concurrence as much as possible
weight_Restaurant = -1

# positive weight, because high school students are good customers
weight_schools = 1

# positive weight, because uni students are good customers
weight_transports = 1.5

# positive weight because employees are even better customers
weight_hotels = 2

In [ ]:
df_weighted = df_data[['Sestiere']].copy()

In [ ]:
df_weighted['Score'] = df_data['Hotels'] * weight_hotels + df_data['Schools'] * weight_schools + df_data['Restaurant'] * weight_Restaurant + df_data['Transport'] * weight_transports
df_weighted = df_weighted.sort_values(by=['Score'], ascending=False)
df_weighted